In [47]:
from rdflib import Graph, Namespace
from pyvis.network import Network
from IPython.display import IFrame, display

# 1. Adjust the path to your TTL file
TTL_FILE = "../linkml/data/rdf/epd_rdf_instance_datastore_canonical_skos_din_bki_shacl.ttl"

# 2. Create and parse the graph
g = Graph()
g.parse(TTL_FILE, format="turtle")

# 3. Define the namespaces you use in your RDF
#    IMPORTANT: Make sure these match exactly the URIs in your TTL
ILCD = Namespace("https://example.org/ilcd/")        # ILCD prefix
DIN  = Namespace("https://example.org/din276/")   # Placeholder for DIN
CC   = Namespace("https://example.org/concreteclass/") 
OBD  = Namespace("https://example.org/obd/") 
BKI  = Namespace("https://example.org/bki/")       # If needed

# 4. Identify EPDs (ProcessDataSet). Adjust the limit as desired.
EPD_LIMIT = 2
q_epds = f"""
SELECT ?epd
WHERE {{
  ?epd a <{ILCD}ProcessDataSet> .
}}
LIMIT {EPD_LIMIT}
"""
results_epd = g.query(q_epds)
epd_uris = [str(row.epd) for row in results_epd]

# ---------------------------------------------------------------------------------------
# Create the PyVis network
# ---------------------------------------------------------------------------------------
net = Network(
    notebook=True, 
    height="1080px", 
    width="1920px", 
    directed=False,
    cdn_resources='in_line'
)

# Optional: add a bit of repulsion to spread nodes out
# (You can tweak these for a looser or tighter layout.)
net.repulsion(
    node_distance=100,     # Default 100
    central_gravity=0.33,  # Default 0.2
    spring_length=200,     # Default 200
    spring_strength=0.05,  # Default 0.05
    damping=0.09           # Default 0.09
)

# ---------------------------------------------------------------------------------------
# For each EPD, gather relationships:
#   1) DIN cost groups
#   2) Concrete strength/weight classifications
#   3) EPD classification data (EPDNorge vs OEKOBAU.DAT; "Ferdig betong," etc.)
# ---------------------------------------------------------------------------------------

epd_number = 0

for epd_uri in epd_uris:
    epd_number += 1
    # ------------------------------------------------------------------
    # A) Add the EPD node
    # ------------------------------------------------------------------
    net.add_node(epd_uri, label=f"EPD {epd_number}", shape="ellipse", color="#a2d2ff")

    # ------------------------------------------------------------------
    # B) Find DIN 276 cost groups
    # ------------------------------------------------------------------
    q_din = f"""
    SELECT ?costgroup
    WHERE {{
      <{epd_uri}> <{DIN}hasDIN276CostGroup> ?costgroup .
    }}
    """
    results_din = g.query(q_din)
    for row in results_din:
        cg_uri = str(row.costgroup)
        cg_label = cg_uri.split("/")[-1]  # or parse as needed
        net.add_node(cg_uri, label=f"DIN {cg_label}", shape="box", color="#fef9c3")
        net.add_edge(epd_uri, cg_uri, label="hasDIN276CostGroup")

    # ------------------------------------------------------------------
    # C) Check compressive strength & weight classification
    # ------------------------------------------------------------------
    q_strength = f"""
    SELECT ?strength
    WHERE {{
      <{epd_uri}> <{CC}hasStrengthClassification> ?strength .
    }}
    """
    strength_results = g.query(q_strength)
    for srow in strength_results:
        strength_uri = str(srow.strength)
        s_label = strength_uri.split("/")[-1]
        net.add_node(strength_uri, label=f"Strength: {s_label}", shape="box", color="#fde2e4")
        net.add_edge(epd_uri, strength_uri, label="hasStrengthClassification")

    q_weight = f"""
    SELECT ?weight
    WHERE {{
      <{epd_uri}> <{CC}hasWeightClassification> ?weight .
    }}
    """
    weight_results = g.query(q_weight)
    for wrow in weight_results:
        weight_uri = str(wrow.weight)
        w_label = weight_uri.split("/")[-1]
        net.add_node(weight_uri, label=f"Weight: {w_label}", shape="box", color="#fde2e4")
        net.add_edge(epd_uri, weight_uri, label="hasWeightClassification")

    # ------------------------------------------------------------------
    # D) Retrieve classification data for each EPD
    #    This shows "EPDNorge" / "OEKOBAU.DAT" classifications, plus entries
    # ------------------------------------------------------------------
    q_classInfo = f"""
    SELECT ?classInfo
    WHERE {{
      <{epd_uri}> <{ILCD}processInformation> ?procInfo .
      ?procInfo <{ILCD}dataSetInformation> ?dataSetInfo .
      ?dataSetInfo <{ILCD}classificationInformation> ?classInfo .
    }}
    """
    cinfo_results = g.query(q_classInfo)

    for ci_row in cinfo_results:
        ci_uri = str(ci_row.classInfo)

        # Add a node for the classificationInformation block (optional)
        net.add_node(ci_uri, label="classificationInformation", shape="ellipse", color="#e7e7e7")
        net.add_edge(epd_uri, ci_uri, label="hasClassificationInfo")

        # Now find the actual classification objects inside classInfo
        q_classifications = f"""
        SELECT ?classification ?className
        WHERE {{
          <{ci_uri}> <{ILCD}classification> ?classification .
          OPTIONAL {{ ?classification <{ILCD}name> ?className . }}
        }}
        """
        sub_results = g.query(q_classifications)
        for sub_row in sub_results:
            cls_uri = str(sub_row.classification)
            cls_name = sub_row.className if sub_row.className else "Classification"
            net.add_node(cls_uri, label=cls_name, shape="box", color="#ccccee")
            net.add_edge(ci_uri, cls_uri, label="classification")

            # Find classification entries
            q_entries = f"""
            SELECT ?entry ?classId ?classVal ?canonCat
            WHERE {{
              <{cls_uri}> <{ILCD}classEntries> ?entry .
              ?entry <{ILCD}classId> ?classId ;
                     <{ILCD}value> ?classVal .
              OPTIONAL {{ ?entry obd:hasCanonicalCategory ?canonCat . }}
            }}
            """
            entry_results = g.query(q_entries)
            for e_row in entry_results:
                entry_uri = str(e_row.entry)
                entry_id  = str(e_row.classId)
                entry_val = str(e_row.classVal)

                # Add node for each classification entry
                node_label = f"{entry_val}"
                net.add_node(entry_uri, label=node_label, shape="ellipse", color="#ffffcc")
                net.add_edge(cls_uri, entry_uri, label="classEntries")

                # If there's a canonical category, link it too
                if e_row.canonCat:
                    canon_uri = str(e_row.canonCat)
                    # For a better label, fetch any skos:prefLabel
                    # We'll do a small sub-query:
                    q_canonLabel = f"""
                    SELECT ?pref
                    WHERE {{
                      <{canon_uri}> skos:prefLabel ?pref .
                      FILTER(lang(?pref) = "en")
                    }}
                    LIMIT 1
                    """
                    canon_label_res = g.query(q_canonLabel)
                    canon_label = None
                    for clrow in canon_label_res:
                        canon_label = str(clrow.pref)
                    if not canon_label:
                        # if no English prefLabel is found, just use URI
                        canon_label = canon_uri.split("/")[-1]

                    net.add_node(canon_uri, label=f"SKOS: {canon_label}", shape="ellipse", color="#ffeedb")
                    net.add_edge(entry_uri, canon_uri, label="hasCanonicalCategory")

# ---------------------------------------------------------------------------------------
# Render and display
# ---------------------------------------------------------------------------------------
html_file = "knowledge_graph.html"
net.show(html_file)

display(IFrame(html_file, width="100%", height="100%"))


knowledge_graph.html
